Bloque de introducción

# Datasets
## Preparación
### Librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Función de descarga
blablabla

In [ ]:
def load_dataset(dataset_url: str, separator: str='\s+', class_position: int=None):
    """Load a dataset from a specified url into a pandas DataFrame.
    
    :param str dataset_url: an url from archive.ics.uci.edu
    :param int class_position: column index where classes are defined (starts by 0)
       if left empty (None), no prediction class will be used (intrinsic case).
    """
    # Load dataset as a pandas DataFrame from a specified url.
    dataset = pd.read_csv(dataset_url, sep=separator, header=None)
    
    # Remove first column as it can be considered an index.
    dataset.drop([0], axis=1, inplace=True)
    
    # Extrinsic case, dataset comes with its classes.
    if class_position is not None:
        # Extract classes.
        classes = dataset.iloc[:, class_position]
        # TODO: aún no sé si dejar o eliminar las clases del dataset,
        # mas que nada para el hue de seaborn a la hora de presentar los datos.
        # Remove classes from the dataset.
        #dataset = dataset.drop([class_position,], axis=1)
        
    # Intrinsic case, dataset has no classes.
    else:
        classes = None
    
    return classes, dataset

### Función de visualización
blablabla    
la idea es que esta función sea parametrizable y que pueda colorear los clusters

In [ ]:
def plot_dataset(dataset: pd.DataFrame, class_position: int=None) -> None:
    # TODO: remove or use the class_position argument.
    #sns.pairplot(dataset, hue=class_position)
    sns.pairplot(dataset)

## Selección

### Dataset extrínseca
blablabla

In [ ]:
#load_dataset()

In [ ]:
#plot_dataset()

blablabla

### Dataset intrínseca
blablabla

In [ ]:
#load_dataset()

In [ ]:
#plot_dataset()

blablabla

# Análisis dataset extrínseca
## Algoritmos

### Algoritmo k-means

### Algoritmo 2

### Algoritmo 3

### Algoritmo 4

### Algritmo 5

## Comparación algoritmos

# Análisis dataset intrínseca¶
## Algoritmos

### Algoritmo k-means

### Algoritmo 2

### Algoritmo 3

### Algoritmo 4

### Algoritmo 5

## Comparación algoritmos

# Conclusión